In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# def read_csv(file, idxname):
#     data = pd.read_csv(file, delimiter="\t", header=None).reset_index().rename(columns = {0: "tokens", "index": idxname}).set_index("tokens")
#     return data

# nfpa_en_vocab = read_csv("/home/ubuntu/cliang/train/pipeline_test_v2/preprocessed_data/NFPA_CS_Train_Vocab.en", "nfpa_idx")
# euro_en_vocab = read_csv("/home/ubuntu/cliang/train/pipeline_test_v2/preprocessed_data/Europarl_Train_Vocab.en", "euro_idx")
# merge_domain_vocab = read_csv("/home/ubuntu/cliang/train/pipeline_test_v2/domain_model/encoder-merge-NFPA_CS_Train_Vocab.en", "merge_idx")

In [5]:
def vocab_exist(token_list, model_dir="/home/ubuntu/cliang/train/pipeline_test_v2", language="en", check_tag = False):
    # check if this word show up in the vocab file
    # :params: model_dir: directory to the model
    # :params: language: the language of the input token list
    # :params: check_tag: if check whether this word belongs to the nfpa/euro vocab, if not, only check whether this word belongs to share vocab
    if check_tag == True:
        merge_vocab = {"en": pd.read_csv(os.path.join(model_dir, "domain_model/encoder-merge-NFPA_CS_Train_Vocab.en"), delimiter="\t", header=None) \
            .rename(columns = {0: "tokens"}).set_index("tokens"), \
                       "es": pd.read_csv(os.path.join(model_dir, "domain_model/decoder-merge-NFPA_CS_Train_Vocab.es"), delimiter="\t", header=None) \
            .rename(columns = {0: "tokens"}).set_index("tokens")}
        nfpa_vocab = {"en": pd.read_csv(os.path.join(model_dir, "preprocessed_data/NFPA_CS_Train_Vocab.en"), delimiter="\t", header=None) \
            .rename(columns = {0: "tokens"}).set_index("tokens"), \
                      "es": pd.read_csv(os.path.join(model_dir, "preprocessed_data/NFPA_CS_Train_Vocab.es"), delimiter="\t", header=None) \
            .rename(columns = {0: "tokens"}).set_index("tokens")}
        euro_vocab = {"en": pd.read_csv(os.path.join(model_dir, "preprocessed_data/Europarl_Train_Vocab.en"), delimiter="\t", header=None) \
            .rename(columns = {0: "tokens"}).set_index("tokens"), \
                      "es": pd.read_csv(os.path.join(model_dir, "preprocessed_data/Europarl_Train_Vocab.es"), delimiter="\t", header=None) \
            .rename(columns = {0: "tokens"}).set_index("tokens")}
        token_tag_dict = {token: (token in merge_vocab[language].index, \
                                  token in nfpa_vocab[language].index, \
                                  token in euro_vocab[language].index) for token in token_list}
        token_tag_dict = {k: "shared" if all([v[1], v[2]]) else ("nfpa" if all([v[1], not v[2]]) else ("euro" if all([not v[1], v[2]]) else None)) for k,v in token_tag_dict.items()}
        token_tag_dict_strip = {k.lstrip("_"): v for (k,v) in token_tag_dict.items()}
        nfpa = [k for k,v in token_tag_dict.items() if v == "nfpa"]
        euro = [k for k,v in token_tag_dict.items() if v == "euro"]
        share = [k for k,v in token_tag_dict.items() if v == "shared"]
        oov = [k for k,v in token_tag_dict.items() if v == None]
        result = {"token_list": token_tag_dict, "striped_token_list": token_tag_dict_strip, "nfpa": nfpa, "euro": euro, "share": share, "oov":oov}
        return result
    else:
        merge_vocab = {"en": pd.read_csv(os.path.join(model_dir, "domain_model/encoder-merge-NFPA_CS_Train_Vocab.en"), delimiter="\t", header=None) \
            .reset_index().rename(columns = {0: "tokens", "index": "merge_idx"}).set_index("tokens"), \
                       "es": pd.read_csv(os.path.join(model_dir, "domain_model/decoder-merge-NFPA_CS_Train_Vocab.es"), delimiter="\t", header=None) \
            .reset_index().rename(columns = {0: "tokens", "index": "merge_idx"}).set_index("tokens")}
        token_tag_dict = {token: token in merge_vocab[language].index for token in token_list}
        token_tag_dict_strip = {k.lstrip("_"): v for (k,v) in token_tag_dict.items()}
        iv = [k for k,v in token_tag_dict.items() if v == True]
        oov = [k for k,v in token_tag_dict.items() if v == False]
        result = {"token_list": token_tag_dict, "striped_token_list": token_tag_dict_strip, "iv": iv, "oov": oov}
        return result

In [8]:
vocab_exist(["this", "is", "575", "fire", "__an", "__apple.", "decorum", "misses"], \
            "/home/ubuntu/cliang/train/pipeline_test_v2", "en", check_tag = False)

{'token_list': {'this': True,
  'is': True,
  '575': True,
  'fire': True,
  '__an': False,
  '__apple.': False,
  'decorum': False,
  'misses': True},
 'striped_token_list': {'this': True,
  'is': True,
  '575': True,
  'fire': True,
  'an': False,
  'apple.': False,
  'decorum': False,
  'misses': True},
 'iv': ['this', 'is', '575', 'fire', 'misses'],
 'oov': ['__an', '__apple.', 'decorum']}

In [9]:
vocab_exist(["this", "is", "575", "fire", "__an", "__apple.", "decorum", "misses"], \
            "/home/ubuntu/cliang/train/pipeline_test_v2", "es", check_tag = True)

{'token_list': {'this': 'shared',
  'is': 'euro',
  '575': 'nfpa',
  'fire': 'shared',
  '__an': None,
  '__apple.': None,
  'decorum': None,
  'misses': None},
 'striped_token_list': {'this': 'shared',
  'is': 'euro',
  '575': 'nfpa',
  'fire': 'shared',
  'an': None,
  'apple.': None,
  'decorum': None,
  'misses': None},
 'nfpa': ['575'],
 'euro': ['is'],
 'share': ['this', 'fire'],
 'oov': ['__an', '__apple.', 'decorum', 'misses']}